In [2]:
import numpy as np 
import pandas as pd 
from selenium import webdriver

## For opening a URL
from urllib.request import urlopen
## For fetching URLs
from urllib.request import Request

"""
HTTP is based on *requests and *responses - the *client makes requests and *servers send responses.
urllib.request mirrors this with a (Request object) which represents the HTTP request you are making.
In its simplest form *you create a (Request object) that specifies the URL you want to fetch. 

Calling *urlopen with this Request object returns a (response object) for the URL requested. 
This response is a file-like object, which means you can for example call .read() on the response
"""



## Parse the html text
from bs4 import BeautifulSoup as soup  

import requests


## 1. Fetch the webpage

In [70]:
### make the link
my_url1 = "https://www.ebay.com/itm/143010095641?hash=item214c10e219:g:XDgAAOSwBQ1df2wn"
my_url2 = "https://www.ebay.com/itm/284151634665?hash=item4228c1dae9:g:bLoAAOSwLVdgBRBK"

my_url = [my_url1, my_url2]

for link in my_url:
    ### Create a Request object that specifies the URL you want to fetch
    req = Request(link, headers={"User-Agent":"Mozilla/5.0"})

    ### urlopen with this Request object returns a (response object) for the URL requested
    webpage = urlopen(req).read()
    #print(webpage)

    ### The *Session object allows you to persist certain parameters across requests
    #with requests.Session() as sess:
        #print(sess)
    clean_webpage = soup(webpage, 'html5lib')

        ### The only difference is that find_all() returns a *list containing the single result, 
        ### and find() just returns the result.
        ### If find_all() can’t find anything, it returns an empty list. 
        ### If find() can’t find anything, it returns None:
    print(clean_webpage.find('span', attrs = {"itemprop" : "price"})['content'])
    #print(clean_webpage)


229.99
455.0


## Google News: Trump

In [55]:
root = "https://www.google.com/"
Url = "https://www.google.com/search?q=trump&sxsrf=AOaemvK_WU3Jg_-9frJCGFlV51FN9HhqbQ:1632400167876&source=lnms&tbm=nws&sa=X&ved=2ahUKEwjMp4zBjJXzAhWNMewKHabzDhEQ_AUoAnoECAEQBA&biw=1536&bih=722&dpr=1.25" 

req = Request(Url, headers={"User-Agent":"Mozilla/5.0"})
page = urlopen(req).read()
#with requests.Session() as sess:
bs = soup(page, 'html5lib')
#print(bs)

## we needed to change the class for item, because class 'kCrYT' was too specific and 
## we needed to use th eparent class of this class.(check if object has this class.)
for item in bs.find_all('div', attrs= { "class":"ZINbbc xpd O9g5cc uUPGi"}):
    #print(item.find('a', href = True)['href'])
    ## we split based on '&sa=U&' beacuse there is a problem by having it in the link 
    ## (when you click on the link, it says page is not available)
    link = (item.find('a', href = True)['href'].split('/url?q=')[1]).split('&sa=U&')[0]
    #print('The link is: {}'.format(link))
    #print("yes")
    #print(item)
    title = item.find('div', attrs = {'class':'BNeawe vvjwJb AP7Wnd'}).get_text()
    ## We replcae (,) with nothing, beuse we wanted to save the information on CSV file
    ## and CSV is sensitive to comma (,)
    title = title.replace(",", "")
    #print('title')
    #print('The title is: {}'.format(title))
    description = item.find('div', attrs = {'class':'BNeawe s3v9rd AP7Wnd'}).get_text()
    ## We replcae (,) with nothing, beuse we wanted to save the information on CSV file
    ## and CSV is sensitive to comma (,)
    description = description.replace(",", "")

    
    #print(title)
    time = description.split(" · ")[0]
    descrpt = description.split(" · ")[1]
    #print('description is: {}'.format(descrpt))
    #print("time is: {}".format(time))

'''
    ## a: read and write the file
    document = open('trump_news.csv', 'a') 
    document.write("{}, {}, {}, {}, \n".format(title, time, descrpt, link))
    document.close()
'''

'\n    ## a: read and write the file\n    document = open(\'trump_news.csv\', \'a\') \n    document.write("{}, {}, {}, {}, \n".format(title, time, descrpt, link))\n    document.close()\n'

## Find the link of defined category in google news


In [66]:
root = "https://news.google.com/"
#Url = "https://www.google.com/search?q=trump&sxsrf=AOaemvK_WU3Jg_-9frJCGFlV51FN9HhqbQ:1632400167876&source=lnms&tbm=nws&sa=X&ved=2ahUKEwjMp4zBjJXzAhWNMewKHabzDhEQ_AUoAnoECAEQBA&biw=1536&bih=722&dpr=1.25" 
new_url = 'https://news.google.com/topstories?hl=en-US&gl=US&ceid=US:en'

req = Request(new_url, headers={"User-Agent":"Mozilla/5.0"})
page = urlopen(req).read()
#with requests.Session() as sess:
bs = soup(page, 'html5lib')
#print(bs)

category = ['World', 'Business', 'Technology', 'Entertainment', 'Sports']
category_link = []
for element in category:
    cat = bs.find('a', attrs = {'aria-label': element})
    #print(category['href'])
    link = root + str(cat['href'])
    category_link.append(link)
    #print(link)
#print(category_link)


################################ search in each page of category ################################
for link in category_link:
    Url = link

    req = Request(Url, headers={"User-Agent":"Mozilla/5.0"})
    page = urlopen(req).read()
    #with requests.Session() as sess:
    bs = soup(page, 'html5lib')
    #print(bs)

    i = 0
    for item in bs.find_all('h3', attrs= { "class":"ipQwMb ekueJc RD0gLb"}):
        if i >= 10:
            break
        else:
            #print("I")
            #print(i)
            first_article = item.find('a', attrs= {'class': "DY5T1d RZIKme"}) # DY5T1d RZIKme
            #print(first_article.getText())
            #print(first_article['href'])
            new_link = root + str(first_article['href'])
            title = first_article.getText()
            title = title.replace(",", "")
            time = bs.find('time', attrs={'class' :"WW6dff uQIVzc Sksgp"}).getText()
            #print(title)
            #print(time)
            i = i + 1


            document = open('news.csv', 'a') 
            document.write("{}, {}, {} \n".format(title, time, new_link))
            document.close()


I
0
I
1
I
2
I
3
I
4
I
5
I
6
I
7
I
8
I
9
I
0
I
1
I
2
I
3
I
4
I
5
I
6
I
7
I
8
I
9
I
0
I
1
I
2
I
3
I
4
I
5
I
6
I
7
I
8
I
9
I
0
I
1
I
2
I
3
I
4
I
5
I
6
I
7
I
8
I
9
I
0
I
1
I
2
I
3
I
4
I
5
I
6
I
7
I
8
I
9
